# OpenStreetMap (San Francisco, CA, USA) Data Wrangling

## Introduction: 
*OpenStreetMap is a map of the world created by people like you and free to use under an open license*

OpenStreetMap is a community based curated map containing data about roads, trails, cafes, train stations, etc. around the world.  Objective of this project is to showcase data wrangling steps when working with large datasets.  My son recently mentioned he wanted to visit San Francisco and I've only been there once myself a long time ago.  Thus, the specific area of focus will be one of America's largest cities, San Francisco, California.

[OpenStreepMap Relation:  San Francisco (111968)](http://www.openstreetmap.org/relation/111968)

[Original San Francisco, California Raw OpenStreetMap dataset from Mapzen](https://mapzen.com/data/metro-extracts/metro/san-francisco_california/)

### It's Data Wrangling Time

In [1]:
import xml.etree.cElementTree as ET
import os
import csv
import codecs
import pprint
import re
import cerberus
import schema
from collections import defaultdict


Let's take a look at how big this dataset is of San Francisco.

In [2]:
ls -l san-francisco_california.osm

 Volume in drive C has no label.
 Volume Serial Number is 6C32-8596

 Directory of C:\Users\ctngu\Google Drive\Udacity\Data Wrangling


 Directory of C:\Users\ctngu\Google Drive\Udacity\Data Wrangling

09/10/2017  02:20 AM     1,410,642,126 san-francisco_california.osm
               1 File(s)  1,410,642,126 bytes
               0 Dir(s)  29,087,682,560 bytes free


Wow, that's a whopping 1.4 GB file.  That's almost 20% the size of my iTunes library.  How about we just take a sample of this dataset, that way doing an overview of the data will take a lot less time trying to go through programmatically.  The following code will go through the original dataset and take every 25th data point and write it into a new file we'll call sample_sf.

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "san-francisco_california.osm"  # Replace this with your osm file
SAMPLE_FILE = "udacity_sample_sf.osm"

k = 125 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):

    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """

    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

The sampled dataset we've created is a lot smaller, from **~1.4 GB** to **~0.057 GB**.  Much easier to work with, especially for my modest laptop.  Would be nice to have a supercomputer though...

In [4]:
ls -l sample_sf.osm

 Volume in drive C has no label.
 Volume Serial Number is 6C32-8596

 Directory of C:\Users\ctngu\Google Drive\Udacity\Data Wrangling


 Directory of C:\Users\ctngu\Google Drive\Udacity\Data Wrangling

09/10/2017  11:36 PM        57,009,542 sample_sf.osm
               1 File(s)     57,009,542 bytes
               0 Dir(s)  29,089,316,864 bytes free


## Time to audit like you bought it.  
*I know, its a pretty lame rhyme...*

Some background info, this datafile is in XML, which is a format that is designed to mark, store, and transport data using "tags" to denote specific information within it.  One thing a person can do to get an idea of the type of information he/she could be working with is by counting the tags, thereby previewing it so to speak.  For the OpenStreetMap, key tags are nodes, ways, and relations. 
 
* **nodes**: consists of a single point in space defined by its latitude, longitude and node id.  In other words, you know your favorite fast-food joint?  Yea, it describes the location of that.  
* **way**: an ordered list of nodes which conceptually serves to describe a road, perimeter or area.  
* **relation**: an ordered list of nodes, ways and/or other relations.  This serves to group these pieces of information by relationships.  For example, think of your home as the starting node and all the places you commonly visit; mall, work, Starbucks, Hot Yoga!, strip ba....yea, all that.  A relation is conceptually grouping related points in space.  
___
### Number of Tags and Tag Types:

In [5]:
def count_tags(filename):
    """
    counts the number of tags within the XML filename and 
    returns a dictionary of tags and the total count of each tag type found.
    """
    tags = {}
    for ev, element in ET.iterparse(filename):
        tag = element.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] = tags[tag]+1
    return tags

In [6]:
count_tags('sample_sf.osm')

{'member': 2696,
 'nd': 311979,
 'node': 265094,
 'osm': 1,
 'relation': 306,
 'tag': 82096,
 'way': 33061}

Clearly quite a lot of places to go to in one of America's largest cities.

* 265,094 nodes
* 33,061 ways
* 206 relations
___
### Tag Checks and Problematic Characters:
Now, going into this dataset one thing we know for sure is that this the OSM data is a community-based project with contributed data provided manually by others like ourselves...Homo Sapiens.  In my life thus far, I've learned, besides blue cheese being the best cheese out there, that we humans are prone to making errors.  So, it wouldn't hurt to check for unexpected characters.

In [7]:
lower = re.compile(r'^([a-z]|_)*$')  # tags that contain ONLY lowercase letters and are valid.
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') # tags that are valid but with a colon.
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') # tags with problematic characters.  

def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        if lower.match(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.match(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1  # other for tags that do not fall into the other three categories.
            
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [8]:
process_map('sample_sf.osm')

{'lower': 57858, 'lower_colon': 23243, 'other': 995, 'problemchars': 0}

This gives us an idea of what kind of characters we can expect in our data values.  Fortunately, our dataset here has **0** tags with problem characters.  Still that doesn't mean we should assume there aren't any unwanted errors in the data.  It's always good to be through.
___
### Unique Users:
Reversing a bit, knowing that OpenStreetMap is a community project I'm actually a bit curious as to exactly who and how many users contributed to this San Francisco dataset.  

In [9]:
def get_user(element):
    return


def process_map(filename):
    """
    Iterate through the OSM file in search for 'user' keys 
    and return a set containing a list of each unique user.
    Return the number of users in list.
    """
    users = set()
    for _, element in ET.iterparse(filename):
        key = 'user'
        if key in element.attrib:
            users.add(element.attrib['user'])
    return users
    

In [10]:
process_map('sample_sf.osm')

{'AndrewBuck',
 'John_Nagle',
 'pluton_od',
 'fproulx',
 'anaris328',
 'mnorelli',
 'Kim Mason',
 'OYR',
 'bballguy',
 'adjuva',
 'KaiRo',
 'CWHz',
 'Sundance',
 'Brian@Brea',
 'sladen',
 'Wim L',
 'posdata',
 'Davidazus',
 'OSMF Redaction Account',
 'bielebog',
 'aguynamedben',
 'RichRico',
 'jorge_o',
 'osmcrc',
 'Colin Jensen',
 'Sarr_Cat',
 'omgar',
 'danrademacher',
 'rowers2',
 'David Speakman',
 'ramyaragupathy',
 'DanDaMan123',
 'jsnake72',
 'nextuniverse',
 'LiisiS',
 'corvy12',
 'Tronikon',
 'allandaly',
 'knockpenny',
 'rgupta0747',
 'Manu1400',
 'MappedReduced',
 'AmanB',
 'Cjlanda',
 'ezekielf',
 'jmtyndall',
 'Field Enterprise',
 'Karen Coyle',
 'BharataHS_sfimport',
 'Warren T',
 'yurasi',
 'Stephen214',
 'lenlo',
 'Jesus arango',
 'dandv',
 'codesliced',
 'ryandrake',
 'Meersbrook',
 'David Strauss',
 'jdhall',
 'California Bear',
 'Bay Area Tango',
 'dirtysouth',
 'Keith Cheveralls',
 'Blaxcarab',
 'Jothirnadh',
 'probabble',
 'volki123',
 'cumbop',
 'edwilcox',
 'Alan

That's a lot of people.  Probably safe to assume they live or have lived in San Francisco.  I've only been to SF once in my life as a wee little tike, so I don't remember much.  Maybe I should hit one of these people up and guide me around town.  (^_^)  
___
### Auditing Street Names:
There are many ways to label addresses; Avenue could be Ave, ave, ave or heck maybe even a full on typo like avenie.  Lets check.  Game plan here is to create a cross check of our street values with a list of expected labels.  If they don't then we'll want to correct them so our data is **consistent**. 

In [11]:
OSMFILE = "sample_sf.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
num_line_street_re = re.compile(r'\d0?(st|nd|rd|th|)\s(Line)$', re.IGNORECASE) # Spell lines ten and under
nth_re = re.compile(r'\d\d?(st|nd|rd|th|)', re.IGNORECASE)
nesw_re = re.compile(r'\s(North|East|South|West)$')


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Crescent", "Gate", "Terrace", "Grove", "Way"]

mapping = { 
            "St": "Street",
            "St.": "Street",
            "st": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Ave,": "Avenue",
            "ave": "Avenue",
            "ave.": "Avenue",
            "ave,": "Avenue",
            "Dr.": "Drive",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Blvd,": "Boulevard",
            "blvd": "Boulevard",
            "blvd.": "Boulevard",
            "blvd,": "Boulevard",
            "Ehs": "EHS",
            "Trl": "Trail",
            "Cir": "Circle",
            "Cir.": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Ctr": "Center",
            "Ctr.": "Center",
            "Ctr,": "Center",
            "ctr": "Center",
            "ctr.": "Center",
            "ctr,": "Center",
            "Crt": "Court",
            "Crt.": "Court",
            "By-pass": "Bypass",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West"
          }

In [12]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    """
    Interate through the OSM file and return a dictionary containing street labels as keys
    and values of street names found with stated street labels.
    """
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping): # name is the actual string of the street, ex:  Keary st
    """(str, dict) -> str
    Check if ending chars of name contain an abbreviation, 
    if True, return updated name replacing abbreviation with matching value in mapping dictionary
    """
    n = street_type_re.search(name)  #this will check to see if there is an abbreviation at the end of name
    if n:  #if True, that is if there is indeed an identified abbreviation...
        street_type = n.group()  # here we will group or segment that abbreviation using group()
        if street_type in mapping: # if that abbreviation is found in our manual mapping
            name = name[:-len(street_type)] + mapping[street_type]  # it will swap it out here with the value in our dict mapping.
    return name

def update_names(audited_file):
    for street_type, ways in audited_file.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name

In [13]:
audited_streets_sf = audit(OSMFILE)

Above we see the dictionary contain the street labels and the street names that contain them.  There are a handful of streets that have their labels abbreviated.  To have our data be uniform, I'm going to opt we use the full name instead of abbreviations, ex:  Blvd -> Boulevard.  This is going to take a bit of extra work but I'll have to review each line of output from above and update my mapping to make sure I catch all street abbreviations I want to be updated.  But once I do, I'll iterate through the OSM file and update the street names.

In [14]:
update_names(audited_streets_sf)

Bridgeway => Bridgeway
10675 => 10675
Fort Mason => Fort Mason
foothill blvd => foothill Boulevard
Kearny st => Kearny Street
Alameda => Alameda
Bayshore Highway => Bayshore Highway
Great Highway => Great Highway
Westlake Center => Westlake Center
El Camino Real => El Camino Real
Cabrillo Highway North => Cabrillo Highway North
South Park => South Park
The Embarcadero => The Embarcadero
Wildwood Gardens => Wildwood Gardens
Indian Rock Path => Indian Rock Path
Monte Verde Dr => Monte Verde Drive
Avenue Del Ora => Avenue Del Ora
Oakridge => Oakridge
Avenue D => Avenue D
Marina Boulevard Building D => Marina Boulevard Building D
Serramonte Ctr => Serramonte Center
United Nations Plaza => United Nations Plaza
El Cerrito Plaza => El Cerrito Plaza
24th St => 24th Street
New Montgomery St => New Montgomery Street
Market St => Market Street
Valencia St => Valencia Street
San Francisco Internation Airport => San Francisco Internation Airport
Broadway => Broadway
Woodside Road, Suite 100 => Wood

___
### Auditing Zipcodes:
Again, doing due-dilligence it wouldn't hurt to check the zip codes for any potential errors.  One thing I learned doing this is that San Francisco's zipcodes all begin in '94'.  And fortunately we can use the code we used to audit the street addresses, albeit with slight variations specific to zipcodes.  

In [15]:
def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if twoDigits != 94 or not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

audited_zipcode_sf = audit_zip(OSMFILE)

Looking at the audited zipcodes, we see for the most part they are all five digit zipcodes starting with 94.  There are a few with extended zipcodes and one with a zipcode for 95498, which isn't San Francisco.  Another had 'CA' mistakenly placed as a zipcode number.  

In [16]:
audited_zipcode_sf

defaultdict(set,
            {'94': {'94002',
              '94010',
              '94014',
              '94015',
              '94019',
              '94025',
              '94038',
              '94044',
              '94061',
              '94063',
              '94065',
              '94066',
              '94070',
              '94080',
              '94102',
              '94103',
              '94104',
              '94105',
              '94107',
              '94108',
              '94109',
              '94110',
              '94111',
              '94112',
              '94113',
              '94114',
              '94115',
              '94116',
              '94117',
              '94118',
              '94118-1316',
              '94121',
              '94122',
              '94122-1515',
              '94123',
              '94124',
              '94127',
              '94128',
              '94129',
              '94131',
              '94132',
              '94133',
 

In [34]:
def update_zip(zipcode):
    """(str) -> str
    Return 5 digit zipcodes.
    """
    zipChar = re.findall('[a-zA-Z]*', zipcode)
    if zipChar:
        zipChar = zipChar[0]
    zipChar.strip()
    if zipChar == "CA":
        updateZip = re.findall(r'\d+', zipcode)
        if updateZip:
            return (re.findall(r'\d+', zipcode))[0]
    else:
        return (re.findall(r'\d+', zipcode))[0]

def update_zips(datafile):
    for street_type, ways in audited_zipcode_sf.iteritems():
        for name in ways:
            better_name = update_zip(name)
            print name, "=>", better_name

In [35]:
update_zips(audited_zipcode_sf)

CA => None
95498 => 95498
94404 => 94404
94401 => 94401
94402 => 94402
94403 => 94403
94118 => 94118
94611 => 94611
94610 => 94610
94613 => 94613
94612 => 94612
94112 => 94112
94113 => 94113
94110 => 94110
94111 => 94111
94116 => 94116
94618 => 94618
94114 => 94114
94115 => 94115
94577 => 94577
94578 => 94578
94579 => 94579
94124 => 94124
94123 => 94123
94122 => 94122
94121 => 94121
94129 => 94129
94128 => 94128
94606 => 94606
94607 => 94607
94605 => 94605
94602 => 94602
94601 => 94601
94044 => 94044
94804 => 94804
94805 => 94805
94608 => 94608
94965 => 94965
94010 => 94010
94131 => 94131
94132 => 94132
94133 => 94133
94134 => 94134
94038 => 94038
94507 => 94507
94501 => 94501
94587 => 94587
94580 => 94580
94303 => 94303
94901 => 94901
94025 => 94025
94596 => 94596
94597 => 94597
94595 => 94595
94598 => 94598
94117 => 94117
94015 => 94015
94014 => 94014
94118-1316 => 94118
94019 => 94019
94523 => 94523
94109 => 94109
94080 => 94080
94002 => 94002
94530 => 94530
94708 => 94708
94541 => 

___
### Preparing, Loading Data into SQL Database:

Now that we've audited and cleaned our data, the next step is to prepare our data into .csv format that will allow it to be imported into a SQL database for further analysis.  

In [ ]:
OSM_PATH = "sample_sf.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def load_new_tag(element, secondary, default_tag_type):
    """
    Load a new tag dict to go into the list of dicts for way_tags, node_tags
    """
    new = {}
    new['id'] = element.attrib['id']
    if ":" not in secondary.attrib['k']:
        new['key'] = secondary.attrib['k']
        new['type'] = default_tag_type

    else:
        post_colon = secondary.attrib['k'].index(":") + 1
        new['key'] = secondary.attrib['k'][post_colon:]
        new['type'] = secondary.attrib['k'][:post_colon - 1]
    new['value'] = secondary.attrib['v']

    print "!23123"
    print secondary.attrib['v']
    print"!2312"
    return new

def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS,
                  problem_chars = PROBLEMCHARS, default_tag_type = 'regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]
        for child in element:
            tag = {}
            if PROBLEMCHARS.search(child.attrib["k"]):
                continue
        
            elif LOWER_COLON.search(child.attrib["k"]):
                tag_type = child.attrib["k"].split(':',1)[0]
                tag_key = child.attrib["k"].split(':',1)[1]
                tag["key"] = tag_key
                if tag_type:
                    tag["type"] = tag_type
                else:
                    tag["type"] = 'regular'
            
                tag["id"] = element.attrib["id"]
                tag["value"] = child.attrib["v"]
            else:
                tag["value"] = child.attrib["v"]
                tag["key"] = child.attrib["k"]
                tag["type"] = "regular"
                tag["id"] = element.attrib["id"]
            if tag:
                tags.append(tag)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for child in element:
            nd = {}
            tag = {}
            if child.tag == 'tag':
                if PROBLEMCHARS.search(child.attrib["k"]):
                    continue
                elif LOWER_COLON.search(child.attrib["k"]):
                    tag_type = child.attrib["k"].split(':',1)[0]
                    tag_key = child.attrib["k"].split(':',1)[1]
                    tag["key"] = tag_key
                    if tag_type:
                        tag["type"] = tag_type
                    else:
                        tag["type"] = 'regular'
                    tag["id"] = element.attrib["id"]
                    tag["value"] = child.attrib["v"]
    
                else:
                    tag["value"] = child.attrib["v"]
                    tag["key"] = child.attrib["k"]
                    tag["type"] = "regular"
                    tag["id"] = element.attrib["id"]
                if tag:
                    tags.append(tag)
                    
            elif child.tag == 'nd':
                nd['id'] = element.attrib["id"]
                nd['node_id'] = child.attrib["ref"]
                nd['position'] = len(way_nodes)
            
                if nd:
                    way_nodes.append(nd)
            else:
                continue
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


In [ ]:
process_map('sample_sf.osm', validate=True)

___
### SQL Queries and Data Exploration:

** File Sizes**

san-francisco_california.osm ..........1,377.6 MB
sample_sf.osm..........................57.7 MB
osm.db.................................40.3 MB
nodes.csv..............................22 MB
nodes_tags.csv.........................385 KB
ways.csv...............................2,008 KB
ways_nodes.csv.........................7,628 KB
ways_tags.csv..........................2,410 KB

**Number of nodes**:

In [ ]:
sqlite> SELECT COUNT(*) FROM Nodes;

265095

**Number of ways**:

In [ ]:
sqlite> SELECT COUNT(*) FROM Ways;

33062

**Number of Unique Users**:

In [ ]:
sqlite> SELECT COUNT(DISTINCT(e.uid))
   ...> FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) e;

1293

**Top 10 Contributing Users**:

In [ ]:
sqlite> SELECT e.user, COUNT(*) as num
   ...> FROM (SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) e
   ...> GROUP BY e.user
   ...> ORDER BY num DESC
   ...> LIMIT(10);

andygol|59833
ediyes|35482
Luis36995|27207
dannykath|21880
RichRico|16534
Rub21|15278
calfarome|7625
oldtopos|6728
KindredCoda|6144
karitotp|5560

** Number of Users Making Only 1 Contribution**:

In [ ]:
sqlite> SELECT COUNT(*) FROM
   ...> (SELECT e.user, COUNT(*) as num
   ...> FROM(SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) e
   ...> GROUP BY e.user
   ...> HAVING num = 1) u;

444

**Top 10 Most Visited Areas**:

In [ ]:
sqlite> SELECT value, COUNT(*) as num
   ...> FROM Nodes_Tags
   ...> WHERE key = 'amenity'
   ...> GROUP BY value
   ...> ORDER BY num DESC
   ...> LIMIT 10;

Which apparently are restaurants.  Make sense...but toilets have only a count of 15?  Something seems off...

restaurant|122
cafe|48
bench|47
post_box|32
place_of_worship|27
fast_food|26
school|25
bicycle_parking|21
toilets|15
bank|14

Which makes me wonder what are the top restaurants?

In [ ]:
sqlite> SELECT Nodes_Tags.value, COUNT(*) as num
   ...> FROM Nodes_Tags
   ...> JOIN (SELECT DISTINCT(id) FROM Nodes_Tags WHERE value = 'restaurant') i
   ...> WHERE Nodes_Tags.id = i.id
   ...> AND Nodes_Tags.key = 'name'
   ...> GROUP BY Nodes_Tags.value
   ...> ORDER BY num DESC
   ...> LIMIT(15);

31st Union|1
ABC Cafe Restaurant|1
Agave Uptown|1
Albany Bowl Cafe|1
American Kitchen|1
Amici's East Coast Pizzeria|1
Arinell Pizza|1
AsiaSF|1
BC Deli Sandwiches|1
BEL|1
Bar Tartine|1
Bistro Burger|1
Britt-Marie's|1
Bubba Gump Shrimp Company|1
Buffalo Wild Wings|1

Interestingly our dataset seems to containly only single values per restaurant, so we can't determine what is the most common eatery.  Which makes me curious because the last I checked San Francisco was a pretty dense city.  So, within our Nodes_Tags table how many actually 'keys' are there with the value 'name'?
___

### Dataset Improvement:

In [ ]:
sqlite> SELECT COUNT(*) FROM Nodes;

265095

In [ ]:
sqlite> SELECT COUNT(*) FROM Nodes_Tags WHERE key = 'name';

741

In [23]:
(741/265095.0) * 100

0.27952243535336385

Only **0.28%** of 'key' values are a name?  Okay, so it's clear that our dataset is indeed lacking on pieces of information that would help improve and provide a more representive map of the city.  To fill in these data gaps, the OpenStreetMap for the San Francisco area could actually pull information from the city's own datasets of the city and county, <https://datasf.org/opendata/>.  Viewing the site, the city has datasets that include geographic locations and boundaries, housings & buildings, and city infrastructure.  We would assume the state has detailed data of itself but there is one caveat to this idea in that because OSM and DATASF are two different databases maintained by different organizations, it should be expected that some tag conventions would be different and would take extra work trying to match the conventions and extract the information.  Fortunately, as we learned in this project, the task could be done programmatically.    
___

## Conclusion:

Overall the San Francisco area dataset seems pretty clean with a few inconsistencies in the zipcode format and street name abbrevations, which shouldn't be surprising for a community-based data project where people have their individual preferences.  With that said, the overall process for data wrangling and cleaning is NOT at all a simple task and takes a lot of time to properly complete.    

## References:
* OSM XML Wiki Documentation:  <https://wiki.openstreetmap.org/wiki/OSM_XML>
* SQL Tutorial:  <https://www.techonthenet.com/sql/index.php>
* Udacity OpenStreetMap Sample Project:   <https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md>
* DATASF: <https://datasf.org/opendata/>
* All the Udacity mentors.  OMG thank you for getting me out of some of them dark black holes of figuring out code.  :D
